In [1]:
import json
import numpy as np
import pandas as pd
import pickle

name2index_file = open("../data/name2index.pkl", "rb")
name2index = pickle.load(name2index_file)

index2name = {index:name for name,index in name2index.items()}

In [3]:
with open('../data/flickr8ktrain.txt') as f:
    train_names = f.readlines()   
with open('../data/flickr8kval.txt') as f:
    val_names = f.readlines()   
with open('../data/flickr8ktest.txt') as f:
    test_names = f.readlines()   
train_id = [name2index[w[0:-1]] for w in train_names]
val_id = [name2index[w[0:-1]] for w in val_names]
test_id = [name2index[w[0:-1]] for w in test_names]

with open('../data/flickr8kzhc.caption.txt') as f:
    chinese_caption = f.readlines()   
with open('../data/flickr8kzhb.caption.txt') as f:
    baidu_caption = f.readlines()   
with open('../data/flickr8kzhg.caption.txt') as f:
    google_caption = f.readlines() 
with open('../data/flickr8kzhmtest.captions.txt') as f:
    en2ch_caption = f.readlines() 

Caption_CN = chinese_caption + en2ch_caption

In [4]:
Caption_CN = [caption for caption in Caption_CN if (caption.split()[0][0:-10] in name2index.keys())]

file_path = '../data/flickr8k_sentences_original.tok.txt'
with open(file_path, 'r') as f:
    Token = f.readlines()  

Caption_infos = []
for i in range(len(Caption_CN)):
    Caption_info = {}
    Caption_info['sentid'] = i
    Caption_info['raw'] = Caption_CN[i].split()[1]
    Caption_info['tokens'] = Token[i].split()
    if Token[i].split()[-1]=='。':
        Caption_info['tokens'] = Token[i].split()[0:-1]
    Caption_info['imgid'] = name2index[Caption_CN[i].split()[0][0:-10]]
    Caption_infos.append(Caption_info)

Caption_infos_imgid = [name2index[caption.split()[0][0:-10]] for caption in Caption_CN]

def get_index1(lst=None, item=''):
     tmp = []
     tag = 0
     for i in lst:
         if i == item:
             tmp.append(tag)
         tag += 1
     return tmp


images = []
for id in train_id+test_id+val_id:
    img_data = {}
    
    img_data['filepath'] = 'flick8kcn_images'
    img_data['imgid'] = id
    img_data['sentids'] = get_index1(Caption_infos_imgid,img_data['imgid'])
    img_data['filename'] = index2name[id]+'.jpg'
    if img_data['imgid'] in train_id:
        img_data['split'] = 'train'
    elif img_data['imgid'] in val_id:
        img_data['split'] = 'val'
    elif img_data['imgid'] in test_id:
        img_data['split'] = 'test'

    img_data['sentences'] = [caption_info for caption_info in Caption_infos if caption_info['imgid'] is img_data['imgid']] 

    img_data['flickr8kcnid'] = img_data['imgid']

    images.append(img_data)

Data_json = {}
Data_json['dataset'] = "flickr8kcn"
Data_json['images'] = images


In [5]:
len(images)

8000

In [6]:
set([len(img_data['sentids']) for img_data in images])

{5, 10}